In [1]:
import os
import random
import time
import json
import warnings 
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import cv2

import numpy as np
import pandas as pd

# 전처리를 위한 라이브러리
from pycocotools.coco import COCO
import torchvision
import torchvision.transforms as transforms

from albumentations.pytorch import ToTensorV2


print('pytorch version: {}'.format(torch.__version__))
print('GPU 사용 가능 여부: {}'.format(torch.cuda.is_available()))

print(torch.cuda.get_device_name(0))
print(torch.cuda.device_count())

# GPU 사용 가능 여부에 따라 device 정보 저장
device = "cuda" if torch.cuda.is_available() else "cpu"

batch_size = 1   # Mini-batch size
# seed 고정
random_seed = 21
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

pytorch version: 1.7.1
GPU 사용 가능 여부: True
Tesla V100-SXM2-32GB
1


In [2]:
# dataset_path  = '/opt/ml/segmentation/input/data'
# anns_file_path = dataset_path + '/' + 'train_all.json'

# # Read annotations
# with open(anns_file_path, 'r') as f:
#     dataset = json.loads(f.read())

# categories = dataset['categories']
# anns = dataset['annotations']
# imgs = dataset['images']
# nr_cats = len(categories)
# nr_annotations = len(anns)
# nr_images = len(imgs)

# # Load categories and super categories
# cat_names = []
# super_cat_names = []
# super_cat_ids = {}
# super_cat_last_name = ''
# nr_super_cats = 0
# for cat_it in categories:
#     cat_names.append(cat_it['name'])
#     super_cat_name = cat_it['supercategory']
#     # Adding new supercat
#     if super_cat_name != super_cat_last_name:
#         super_cat_names.append(super_cat_name)
#         super_cat_ids[super_cat_name] = nr_super_cats
#         super_cat_last_name = super_cat_name
#         nr_super_cats += 1

# print('Number of super categories:', nr_super_cats)
# print('Number of categories:', nr_cats)
# print('Number of annotations:', nr_annotations)
# print('Number of images:', nr_images)

Number of super categories: 10
Number of categories: 10
Number of annotations: 26240
Number of images: 3272


In [3]:
# # Count annotations
# cat_histogram = np.zeros(nr_cats,dtype=int)
# for ann in anns:
#     cat_histogram[ann['category_id']-1] += 1

# # Convert to DataFrame
# df = pd.DataFrame({'Categories': cat_names, 'Number of annotations': cat_histogram})
# df = df.sort_values('Number of annotations', 0, False)

In [4]:
# # category labeling 
# sorted_temp_df = df.sort_index()

# # background = 0 에 해당되는 label 추가 후 기존들을 모두 label + 1 로 설정
# sorted_df = pd.DataFrame(["Backgroud"], columns = ["Categories"])
# sorted_df = sorted_df.append(sorted_temp_df, ignore_index=True)

In [25]:
sorted_df

,Categories,Number of annotations
0,Backgroud,NaN
1,General trash,2782.0
2,Paper,9311.0
3,Paper pack,659.0
4,Metal,562.0
5,Glass,610.0
6,Plastic,3090.0
7,Styrofoam,1343.0
8,Plastic bag,7643.0
9,Battery,63.0


In [2]:
# category_names = list(sorted_df.Categories)
category_names = ("Backgroud", "General trash", "Paper", "Paper pack",
                    "Metal", "Glass", "Plastic", "Styrofoam",
                    "Plastic bag", "Battery", "Clothing")
def get_classname(classID, cats):
    for i in range(len(cats)):
        if cats[i]['id']==classID:
            return cats[i]['name']
    return "None"

class CustomDataLoader(Dataset):
    """COCO format"""
    def __init__(self, data_dir, mode = 'train', transform = None):
        super().__init__()
        self.mode = mode
        self.transform = transform
        self.coco = COCO(data_dir)
        
    def __getitem__(self, index: int):
        # dataset이 index되어 list처럼 동작
        image_id = self.coco.getImgIds(imgIds=index)
        image_infos = self.coco.loadImgs(image_id)[0]
        
        # cv2 를 활용하여 image 불러오기
        images = cv2.imread(os.path.join(dataset_path, image_infos['file_name']))
        images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB).astype(np.float32)
        images /= 255.0
        
        if (self.mode in ('train', 'val')):
            ann_ids = self.coco.getAnnIds(imgIds=image_infos['id'])
            anns = self.coco.loadAnns(ann_ids)

            # Load the categories in a variable
            cat_ids = self.coco.getCatIds()
            cats = self.coco.loadCats(cat_ids)

            # masks : size가 (height x width)인 2D
            # 각각의 pixel 값에는 "category id" 할당
            # Background = 0
            masks = np.zeros((image_infos["height"], image_infos["width"]))
            # General trash = 1, ... , Cigarette = 10
            anns = sorted(anns, key=lambda idx : len(idx['segmentation'][0]), reverse=False)
            for i in range(len(anns)):
                className = get_classname(anns[i]['category_id'], cats)
                pixel_value = category_names.index(className)
                masks[self.coco.annToMask(anns[i]) == 1] = pixel_value
            masks = masks.astype(np.int8)
                        
            # transform -> albumentations 라이브러리 활용
            if self.transform is not None:
                transformed = self.transform(image=images, mask=masks)
                images = transformed["image"]
                masks = transformed["mask"]
            return images, masks, image_infos
        
        if self.mode == 'test':
            # transform -> albumentations 라이브러리 활용
            if self.transform is not None:
                transformed = self.transform(image=images)
                images = transformed["image"]
            return images, image_infos
    
    def __len__(self) -> int:
        # 전체 dataset의 size를 return
        return len(self.coco.getImgIds())

In [5]:
# train.json / validation.json / test.json 디렉토리 설정
dataset_path  = '/opt/ml/segmentation/input/data'
train_path = dataset_path + '/train.json'
val_path = dataset_path + '/val.json'
test_path = dataset_path + '/test.json'


# collate_fn needs for batch
def collate_fn(batch):
    return tuple(zip(*batch))


import albumentations as A
from albumentations.pytorch import ToTensorV2

train_transform = A.Compose([
                            ToTensorV2()
                            ])

val_transform = A.Compose([
                          ToTensorV2()
                          ])

test_transform = A.Compose([
                           ToTensorV2()
                           ])

# create own Dataset 1 (skip)
# validation set을 직접 나누고 싶은 경우
# random_split 사용하여 data set을 8:2 로 분할
# train_size = int(0.8*len(dataset))
# val_size = int(len(dataset)-train_size)
# dataset = CustomDataLoader(data_dir=train_path, mode='train', transform=transform)
# train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# create own Dataset 2
# train dataset
train_dataset = CustomDataLoader(data_dir=train_path, mode='train', transform=train_transform)

# validation dataset
val_dataset = CustomDataLoader(data_dir=val_path, mode='val', transform=val_transform)

# test dataset
test_dataset = CustomDataLoader(data_dir=test_path, mode='test', transform=test_transform)


# DataLoader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size,
                                           shuffle=False,
                                           num_workers=4,
                                           collate_fn=collate_fn)

val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                         batch_size=batch_size,
                                         shuffle=False,
                                         num_workers=4,
                                         collate_fn=collate_fn)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          num_workers=4,
                                          collate_fn=collate_fn)

loading annotations into memory...
Done (t=3.59s)
creating index...
index created!
loading annotations into memory...
Done (t=1.26s)
creating index...
index created!
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


In [6]:
import shutil 


In [12]:
for imgs, masks, image_infos in train_loader:
    print(image_infos)
    break

({'license': 0, 'url': None, 'file_name': 'batch_01_vt/0003.jpg', 'height': 512, 'width': 512, 'date_captured': None, 'id': 0},)


In [16]:
# for imgs, masks, image_infos in train_loader:
#     image_infos = image_infos[0]
#     file_dir = f"/opt/ml/semantic-segmentation-level2-cv-07/input/mmseg/ann_dir/training/{image_infos['id']:04}.png"
#     src_img= f"/opt/ml/semantic-segmentation-level2-cv-07/input/data/{image_infos['file_name']}"
#     trg_img=f"/opt/ml/semantic-segmentation-level2-cv-07/input/mmseg/img_dir/training/{image_infos['id']:04}.jpg"
#     shutil.move(src_img, trg_img)
#     masks = masks[0].numpy()
#     cv2.imwrite(file_dir, masks)
    
for imgs, masks, image_infos in val_loader:
    image_infos = image_infos[0]
    # file_dir = f"/opt/ml/semantic-segmentation-level2-cv-07/input/mmseg/ann_dir/validation/{image_infos['id']:04}.png"
    src_img= f"/opt/ml/semantic-segmentation-level2-cv-07/input/data/{image_infos['file_name']}"
    trg_img=f"/opt/ml/semantic-segmentation-level2-cv-07/input/mmseg/img_dir/validation/{image_infos['id']:04}.jpg"
    
    shutil.move(src_img, trg_img)
    # masks = masks[0].numpy()
    # cv2.imwrite(file_dir, masks)
    


In [10]:
for imgs, image_infos in test_loader:
    image_infos = image_infos[0]
    src_img= f"/opt/ml/semantic-segmentation-level2-cv-07/input/data/{image_infos['file_name']}"
    trg_img=f"/opt/ml/semantic-segmentation-level2-cv-07/input/mmseg/img_dir/test/{image_infos['id']:04}.jpg"
    shutil.move(src_img, trg_img)